In [1]:
import ipywidgets as widgets
from IPython.display import display

from gridlab import Action, ViewPipeline, create_world, verify_all_solutions
from gridlab.view import HTMLTableLegendView, HTMLTableStatusView, HTMLGridView

In [2]:
STYLE  = """
.legend tr, .status tr, .grid tr {
    background-color: #ffffff;
}

.legend td, .status  td {
   border: 1px solid #000000;
}

.legend td.entity {
    font-family: 'Source Code Pro', monospace;
    text-align: center;
}

.status td.status-key {
    font-weight: bold;
}

.grid td {
    font-family: 'Source Code Pro', monospace;
    font-size: 24px;
    text-align: center;
    width: 1.5em;
    height: 1.5em;
}

.grid span {
    font-family: 'Source Code Pro', monospace;
    font-size: 24px;
    line-height: 1em;
}
"""

In [11]:
class GridWorldWidget:
    def __init__(self, pipeline: ViewPipeline, world_name: str = 'causeway'):
        self.name = world_name
        self.world = None
        self.views = {}
        self.pipeline = pipeline

        def make_option(name: str, ok: bool):
            m = '✅' if ok else '⛔️'
            return (f'{m} {name}', name)

        solution_status_dict = verify_all_solutions(report=False)
        solution_status_items = sorted(solution_status_dict.items(), key=lambda kv: not kv[1][0])
        dropdown_data = {
            'options': [make_option(name, ok) for name, (ok, _) in solution_status_items],
            'description': 'Select world:',
            'value': world_name,
        }

        # Widgets for display
        self.world_dropdown = widgets.Dropdown(**dropdown_data)
        self.legend_display = widgets.HTML(self._format_legend())
        self.status_display = widgets.HTML(self._format_status())
        self.grid_display = widgets.HTML(self._format_grid())

        # Control buttons
        self.up_button = widgets.Button(description='⏶', layout=widgets.Layout(width='40px'))
        self.down_button = widgets.Button(description='⏷', layout=widgets.Layout(width='40px'))
        self.left_button = widgets.Button(description='⏴', layout=widgets.Layout(width='40px'))
        self.right_button = widgets.Button(description='⏵', layout=widgets.Layout(width='40px'))
        self.none_button = widgets.Button(description='∅', layout=widgets.Layout(width='40px'))
        self.reset_button = widgets.Button(description='↻', layout=widgets.Layout(width='40px'))

        # Callbacks
        self.world_dropdown.observe(self._on_world_selected)
        self.up_button.on_click(lambda _: self._on_move(Action.UP))
        self.down_button.on_click(lambda _: self._on_move(Action.DOWN))
        self.left_button.on_click(lambda _: self._on_move(Action.LEFT))
        self.right_button.on_click(lambda _: self._on_move(Action.RIGHT))
        self.none_button.on_click(lambda _: self._on_move(Action.NONE))
        self.reset_button.on_click(lambda _: self._initialize())

        # Layout
        controls_top = widgets.HBox(
            [self.up_button],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
            ),
        )
        controls_middle = widgets.HBox(
            [self.left_button, self.none_button, self.right_button],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
            ),
        )
        controls_bottom = widgets.HBox(
            [self.down_button],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
            ),
        )
        controls_reset = widgets.HBox(
            [self.reset_button],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
            ),
        )
        controls = widgets.VBox(
            [controls_top, controls_middle, controls_bottom, controls_reset],
            layout=widgets.Layout(padding='20px'),
        )

        page = widgets.VBox(
            [
                self.world_dropdown,
                self.legend_display,
                self.status_display,
                self.grid_display,
                controls,
            ],
        )

        wrapper = widgets.HBox([page], layout=widgets.Layout(padding='20px'))

        self._initialize()
        display(wrapper)

    def _initialize(self):
        if self.name is None:
            return

        self.world = create_world(self.name)
        self.views = self.pipeline.render(self.world)
        self._refresh_view()

    def _format_legend(self):
        if self.world is None:
            return ''

        theme_style = self.pipeline.theme.css()
        style = f'{STYLE}\n\n{theme_style}'
        legend = self.views['legend']
        return f'<style>{style}</style><div><b>Legend</b>{legend}</div>'

    def _format_status(self):
        if self.world is None:
            return ''

        status = self.views['status']
        return f'<div><b>Status</b>{status}</div>'

    def _format_grid(self):
        if self.world is None:
            return ''

        grid = self.views['grid']
        return f'<div><b>Grid</b>{grid}</div>'

    def _on_world_selected(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.name = change['new']
            self._initialize()

    def _on_move(self, action: Action):
        if self.world is None:
            return

        self.world.step(action=action)
        self._refresh_view()

    def _refresh_view(self):
        self.views = self.pipeline.render(self.world)

        # Refresh displays in-place
        self.legend_display.value = self._format_legend()
        self.status_display.value = self._format_status()
        self.grid_display.value = self._format_grid()

In [ ]:
def run():
    pipeline = ViewPipeline(
        {
            'legend': HTMLTableLegendView(),
            'status': HTMLTableStatusView(),
            'grid': HTMLGridView(),
        },
        theme='desert',
    )
    _ = GridWorldWidget(pipeline, world_name='causway')

run()